In [31]:
from binance.client import Client as bnb_client
from datetime import datetime
import pandas as pd 
from datetime import datetime, timezone
from itertools import product

In [32]:
# Downloading Data 

client = bnb_client(tld='US')

def get_binance_px(symbol, freq, start_ts = '2020-01-01', end_ts='2024-06-30'):
    data = client.get_historical_klines(symbol, freq, start_ts, end_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    data['open_time'] = data['open_time'].map(lambda x: datetime.fromtimestamp(x/1000, timezone.utc))
    data['close_time'] = data['close_time'].map(lambda x: datetime.fromtimestamp(x/1000, timezone.utc))
    return data 

univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','DOTUSDT', 'MATICUSDT']
start_ts = '2020-01-01'
end_ts = '2024-06-30'

columns = pd.MultiIndex.from_tuples(product(('Close', 'Vol'), univ), names=['Category', 'Asset'])

freq = '1h'
px = pd.DataFrame(columns=columns) 
for x in univ:
    df = get_binance_px(x, freq, start_ts, end_ts).set_index('open_time')
    px['Close', x] = df['close'].astype(float)
    px['Vol', x] = df['volume'].astype(float)

freq = '1d'
px_w = pd.DataFrame(columns=columns) 
for x in univ:
    df = get_binance_px(x, freq, start_ts, end_ts).set_index('open_time')
    px_w['Close', x] = df['close'].astype(float)
    px_w['Vol', x] = df['volume'].astype(float)

In [33]:
px_filename = '../../data/AmirAlamir/1h_2020-01-01_2024-06-30'
px_w_filename = '../../data/AmirAlamir/1d_2020-01-01_2024-06-30'

px.to_csv(px_filename + '.csv')
px_w.to_csv(px_w_filename + '.csv')

px.to_pickle(px_filename + '.pkl')
px_w.to_pickle(px_w_filename + '.pkl')